# Classificação das notas de filme - IMDb

IMDb é uma base de dados online com informações relativas a filmes e programas de TV. 
No site, podem ser encontradas avaliações de filmes e as suas respectivas notas.

Com base nesse dado, é possível criar um modelo capaz de, dado uma crítica/avalição do filme, sabermos sua avaliação foi positiva ou negativa.

O dataset e informações adicionais podem ser encontrados nesse link também:
https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

### Bibliotecas

In [ ]:
!pip install unidecode
!pip install tqdm==4.33.0

     |████████████████████████████████| 245kB 2.9MB/s 
     |████████████████████████████████| 51kB 1.7MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import unidecode
import re
import string
import nltk
from sklearn.linear_model import PassiveAggressiveClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
sns.set()
tqdm.pandas()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Leitura dos dados

In [ ]:
df = pd.read_csv('IMDB Dataset.csv')

In [ ]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
df.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

O dado está dividido em exatamente 25000 samples negativos e 25000 positivos. Não há dados faltantes também. Então não há preocupação em balanceamento dos dados ou necessidade de under/oversampling.

In [ ]:
def get_vocab_size(corpus):
    vocab = set()
    for review in corpus:
        for word in review.split():
            vocab.add(word)
    return vocab

In [ ]:
print('Tamanho do vocabulário antes do pré-processamento:', len(get_vocab_size(df.review)))


Tamanho do vocabulário antes do pré-processamento: 438729


### Pré-processamento

Como se pôde observar, há tags HTML inseridas no texto. Primeiramente, elas serão retiradas.


In [ ]:
HTML_REGEX = re.compile(r'<.*?>')

def remove_html_tags(text):    
    text = re.sub(HTML_REGEX, '', text)
    return text


In [ ]:
df['cleaned_review'] = df.review.apply(remove_html_tags)
df

,review,sentiment,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,A wonderful little production. The filming tec...
2,I thought this was a wonderful way to spend ti...,positive,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"Petter Mattei's ""Love in the Time of Money"" is..."
...,...,...,...
49995,I thought this movie did a down right good job...,positive,I thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...,negative,I am a Catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...,negative,I'm going to have to disagree with the previou...


O próximo passo é realizar o restante do pré-processamento do texto, passando o texto para lowercase, removendo símbolos, pontuações e acentuação e stopwords. O objetivo dessa fase é remover ruído do corpus assim como palavras que possuem uma frequência alta na língua inglesa apesar de possuir pouca influência na semântica.


In [ ]:
STOPWORDS = set(stopwords.words('portuguese'))

# regex para remove todos caracteres, exceto letras, números e espaços
WORDS_AND_NUMBERS_REGEX = re.compile(r'[^a-zA-Z0-9 ]+')

def pre_process(text):
    # Texto para lowercase
    text = text.lower()

    # Remover acentos
    text = unidecode.unidecode(text)

    # Remover símbolos e pontuação
    text = re.sub(WORDS_AND_NUMBERS_REGEX, '', text)

    # Remover stopwords
    text = text.split()
    text = [word for word in text if word not in STOPWORDS]

    return ' '.join(text)



In [ ]:
pre_process("Ser ruim em alguma coisa é o primeiro passo para se tornar bom em alguma coisa.")

'ser ruim alguma coisa primeiro passo tornar bom alguma coisa'

In [ ]:
df['cleaned_review'] = df.cleaned_review.apply(pre_process)
df

,review,sentiment,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching 1 oz episode ...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically theres family little boy jake thinks...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,thought movie right good job wasnt creative or...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,negative,catholic taught parochial elementary schools n...
49998,I'm going to have to disagree with the previou...,negative,im going disagree previous comment side maltin...


In [ ]:
print('Tamanho do vocabulário após pré-processamento inicial:', len(get_vocab_size(df.cleaned_review)))


Tamanho do vocabulário após pré-processamento inicial: 221408


O próximo passo é realizar os processos de lemmatization e stemming. Eles também têm o objetivo de tirar ruído do texto, mas com a intenção de agrupar termos similares. O primeiro se refere a agrupar variações de uma mesma palavra ou sinônimos em um mesmo termo, como bons e boa em bom. O segundo se trata do mantimento do radical das palavras, como em corrida e corro, que virariam "corr".

In [ ]:
!python -m spacy download pt_core_news_sm


     |████████████████████████████████| 12.9MB 845kB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.1.0-cp36-none-any.whl size=12843677 sha256=bdbe04d39468c06487b4b1413494bcd4e65ab054f1fc22123f39e2c3d0d49e8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-fwrq4e89/wheels/a3/8f/c1/f036e3a7f1aa44fb06a534c6c4b1c2b773f101fdb1f163c08c
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [ ]:
import spacy

nlp = spacy.load('pt_core_news_sm')

In [ ]:
a = nlp("tudo que é pequeno é só uma versão menor de algo grande")

In [ ]:
STOPWORDS

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houveríamos',
 'houvesse',


#### Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

def lemmatize(text):
  text = text.split()
  lemmatized_text = []
  for word in text:
      lemmatized_text.append(lemmatizer.lemmatize(word))
  return ' '.join(lemmatized_text)




In [ ]:
df["cleaned_review"] = df.cleaned_review.progress_apply(lemmatize)
df

100%|██████████| 50000/50000 [00:22<00:00, 2204.34it/s]


,review,sentiment,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching 1 oz episode y...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically there family little boy jake think t...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,thought movie right good job wasnt creative or...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,negative,catholic taught parochial elementary school nu...
49998,I'm going to have to disagree with the previou...,negative,im going disagree previous comment side maltin...


In [ ]:
print('Tamanho do vocabulário após lemmatization:', len(get_vocab_size(df.cleaned_review)))


Tamanho do vocabulário após lemmatization: 210228


#### Stemming

In [ ]:
from nltk.stem import PorterStemmer 

stemmer = PorterStemmer() 

def stem(text):
    text = text.split()
    stemmed_text = []
    for word in text:
        stemmed_text.append(stemmer.stem(word))
    return ' '.join(stemmed_text)


In [ ]:
df["cleaned_review"] = df.cleaned_review.progress_apply(stem)
df

100%|██████████| 50000/50000 [01:46<00:00, 471.31it/s]


,review,sentiment,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,one review mention watch 1 oz episod youll hoo...
1,A wonderful little production. <br /><br />The...,positive,wonder littl product film techniqu unassum old...
2,I thought this was a wonderful way to spend ti...,positive,thought wonder way spend time hot summer weeke...
3,Basically there's a family where a little boy ...,negative,basic there famili littl boy jake think there ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei love time money visual stun film...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,thought movi right good job wasnt creativ orig...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogu bad act idiot direct anno...
49997,I am a Catholic taught in parochial elementary...,negative,cathol taught parochi elementari school nun ta...
49998,I'm going to have to disagree with the previou...,negative,im go disagre previou comment side maltin one ...


In [ ]:
print('Tamanho do vocabulário após stemming:', len(get_vocab_size(df.cleaned_review)))


Tamanho do vocabulário após stemming: 181191


### Split

In [ ]:
from sklearn.model_selection import train_test_split

X, y = df.cleaned_review, df.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=0.2)

### Feature extraction

Para extração de features, serão testados três algoritmos: Tf-Idf, Hashing e Doc2Vec. Os dois primeiros se baseam apenas na frequência dos termos ao longo dos documentos para criar vetores. O terceiro utiliza aprendizado supervisionado com o objetivo de criar vetores semelhantes para documentos semelhantes.

#### Tf-Idf Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer().fit(X_train)
train_tfidf = tfidf.transform(X_train)
test_tfidf = tfidf.transform(X_test)

#### Hashing Vectorizer

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

train_hv = HashingVectorizer(n_features=2**18).fit_transform(X_train)
test_hv = HashingVectorizer(n_features=2**18).fit_transform(X_test)

#### Doc2Vec

In [ ]:
from nltk.tokenize import word_tokenize

corpus = [review for review in df.cleaned_review.progress_apply(word_tokenize)]

100%|██████████| 50000/50000 [00:26<00:00, 1917.97it/s]


In [ ]:
len(corpus)

50000

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_corpus = [TaggedDocument(words=words, tags=[str(i)]) for i, words in enumerate(corpus)]
model = Doc2Vec(tagged_corpus, window=3, vector_size=300, min_count=1, workers=4)

In [ ]:
train_d2v = np.array([model.docvecs[ix] for ix in X_train.index])
test_d2v = np.array([model.docvecs[ix] for ix in X_test.index])

### Treinamento

No treinamento, serão usados 3 tipos de classificadores:
* PassiveAgressive Classifier, um classificador bastante simples, com um funcionamento similar a Máquinas de Vetores de Suporte, porém com o diferencial de permitir ajustar sua agressividade (daí o nome) para ajustar o vetor de pesos em caso de uma classificação errada;
* XGBoost, um modelo que funciona a partir de ensemble de árvores e Gradient Boosting, que basicamente tem o objetivo de, a partir de classificadores mais simples e "fracos" (weak "learners"), um classificador complexo e mais geral pode ser criado;
* CNN, um tipo de rede neural normalmente usada em imagens, mas com aplicações para NLP também.

#### PassiveAgressive Classifier

##### Tf-Idf

In [ ]:
pac = PassiveAggressiveClassifier(random_state=42).fit(train_tfidf, y_train)
y_pred_pac_tfidf = pac.predict(test_tfidf)
print(classification_report(y_test, y_pred_pac_tfidf))

              precision    recall  f1-score   support

    negative       0.88      0.87      0.88      5000
    positive       0.87      0.89      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



##### Hashing

In [ ]:
pac_hv = PassiveAggressiveClassifier(random_state=42).fit(train_hv, y_train)
y_pred_pac_hv = pac_hv.predict(test_hv)
print(classification_report(y_test, y_pred_pac_hv))

              precision    recall  f1-score   support

    negative       0.88      0.87      0.87      5000
    positive       0.87      0.88      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



##### Doc2Vec

In [ ]:
pac_d2v = PassiveAggressiveClassifier(random_state=42).fit(train_d2v, y_train)
y_pred_pac_d2v = pac_d2v.predict(test_d2v)
print(classification_report(y_test, y_pred_pac_d2v))

              precision    recall  f1-score   support

    negative       0.89      0.63      0.74      5000
    positive       0.71      0.92      0.81      5000

    accuracy                           0.78     10000
   macro avg       0.80      0.78      0.77     10000
weighted avg       0.80      0.78      0.77     10000



#### XGBoost

##### Tf-Idf

In [ ]:
xgb = XGBClassifier(max_depth=10, random_state=42).fit(train_tfidf, y_train)
y_pred_xgb_tfidf = xgb.predict(test_tfidf)
print(classification_report(y_test, y_pred_xgb_tfidf))

              precision    recall  f1-score   support

    negative       0.85      0.82      0.84      5000
    positive       0.83      0.86      0.84      5000

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



##### Hashing

In [ ]:
xgb_hv = XGBClassifier(max_depth=10, random_state=42).fit(train_hv, y_train)
y_pred_xgb_hv = xgb_hv.predict(test_hv)
print(classification_report(y_test, y_pred_xgb_hv))

              precision    recall  f1-score   support

    negative       0.86      0.82      0.84      5000
    positive       0.83      0.86      0.84      5000

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



##### Doc2Vec

In [ ]:
xgb_d2v = XGBClassifier(max_depth=10, random_state=42).fit(train_d2v, y_train)
y_pred_xgb_d2v = xgb_d2v.predict(test_d2v)
print(classification_report(y_test, y_pred_xgb_d2v))

              precision    recall  f1-score   support

    negative       0.82      0.81      0.81      5000
    positive       0.81      0.82      0.82      5000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



#### CNN

In [ ]:
tf.keras.backend.clear_session()

Para a CNN, o vetor de entrada será criado por meio do Tokenizer oferecido pelo Keras. Ele dá um valor único para cada token presente e por meio do método text_to_sequences, transforma cada documento numa sequência desses identificadores.

In [ ]:
tokenizer = Tokenizer(num_words=182000)
tokenizer.fit_on_texts(X_train)

train_cnn_t2s = tokenizer.texts_to_sequences(X_train)
test_cnn_t2s = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

In [ ]:
maxlen = 100

train_cnn_t2s = pad_sequences(train_cnn_t2s, padding='post', maxlen=maxlen)
test_cnn_t2s = pad_sequences(test_cnn_t2s, padding='post', maxlen=maxlen)

In [ ]:
y_train_cnn = [1 if i == "positive"  else 0 for i in y_train]
y_test_cnn = [1 if i == "positive"  else 0 for i in y_test]

In [ ]:
# maxlen -> Nº de dimensões dos vetor de entrada gerado após tokenizer + pad_seq
# vocab_size -> Tamanho do vocabulário, ou seja, número de palavras distintas
# embedding_dim -> Nº de dimensões do vetor de output da camada de Embedding

embedding_dim = 300

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          46569300  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 96, 128)           192128    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 94, 128)           49280     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 46,819,029
Trainable params: 46,819,029
Non-trainable params: 0
__________________________________________

In [ ]:
history = model.fit(train_cnn_t2s, y_train_cnn,
                    epochs=10,
                    validation_data=(test_cnn_t2s, y_test_cnn),
                    batch_size=16)

loss, accuracy = model.evaluate(train_cnn_t2s, y_train_cnn, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_cnn_t2s, y_test_cnn, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 70s 2ms/sample - loss: 0.3701 - acc: 0.8318 - val_loss: 0.2830 - val_acc: 0.8803
Epoch 2/10
40000/40000 [==============================] - 71s 2ms/sample - loss: 0.1591 - acc: 0.9386 - val_loss: 0.3527 - val_acc: 0.8577
Epoch 3/10
40000/40000 [==============================] - 71s 2ms/sample - loss: 0.0478 - acc: 0.9833 - val_loss: 0.3921 - val_acc: 0.8722
Epoch 4/10
40000/40000 [==============================] - 71s 2ms/sample - loss: 0.0238 - acc: 0.9919 - val_loss: 0.5155 - val_acc: 0.8642
Epoch 5/10
40000/40000 [==============================] - 70s 2ms/sample - loss: 0.0165 - acc: 0.9943 - val_loss: 0.7627 - val_acc: 0.8595
Epoch 6/10
40000/40000 [==============================] - 70s 2ms/sample - loss: 0.0127 - acc: 0.9956 - val_loss: 0.7893 - val_acc: 0.8622
Epoch 7/10
40000/40000 [==============================] - 70s 2ms/sample - loss: 0.0087 - acc: 0.9971 - val_loss: 0.

# Conclusão

O modelo que melhor conseguiu classificar as avaliações, tomando como base a acurácia, foi a CNN, com quase 86% no dataset de treino. Como o dado é distribuido igualmente entre as duas classes ("positive" e "negative"), essa métrica nos dá um bom parâmetro da capacidade do modelo

Esse resultado nos permite inferir que, na teoria, se 100 novas avaliações fossem apresentados ao modelo, ele conseguiria determinar em quase 86 dos casos a classificação correta da avaliação. Nos outros 14%, ocorreria o que chamamos de erros do tipo I e tipo II. 

Em classificações binárias, como o caso desse notebook, erros do tipo I, também chamados de falso positivo, ocorrem quando uma amostra que é da classe negativa é dita como pertencente da classe positiva, ou seja, um filme que foi avaliado como ruim é considerado bom pelo modelo.

Já os erros do tipo II (falsos negativos) representam a situação inversa, isto é, um filme bom é visto como ruim.